# Regrouping knowledge in an automatized way for binary classification


Starting point:
X and Y are formatted



In [7]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import os
import matplotlib as plt

import sklearn as sk
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
import numpy as np
import xgboost as xgb 


import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('Data/titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
### get the inner join of sales and controls


X = df.drop('Survived', axis =1)
Y = df['Survived']
hypo = X.columns
test = Y.columns


X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25, random_state=42)


## Imbalance:
#### Undersampling

In [24]:

def undersample(X_train,Y_train, target_var = str, minor_class = 1, limit_sample = 200):
    """
    Simple undersampling, test keep the same proportion
    
    """
    # Rebuild training set
    training_set = pd.concat([X_train,Y_train],axis = 1)
    
    # Choose major class, and minor class as 1 and 0s
    major_class = 1-minor_class
    print("Previous split : \n0 : ",training_set[training_set[target_var] == 0].shape[0],"\n1 : ",
          training_set[training_set[target_var] == 1].shape[0])
    
    
    #Define which values will be removed randomly
    major_indices  = training_set[training_set[target_var] == major_class].index
    minor_indices = training_set[training_set[target_var] == minor_class].index
    random_indices = np.random.choice(major_indices, limit_sample, replace=False)
    kept_indices = np.sort(np.append(minor_indices,random_indices))

                             
    #Redefine the training set according to the undersampling
    training_set_undersampled  = training_set.loc[kept_indices]


    print("New split : \n0 : ",training_set_undersampled[training_set_undersampled[target_var] == 0].shape[0],"\n1 : ",
          training_set_undersampled[training_set_undersampled[target_var] == 1].shape[0])
    

    training_set_undersampled
    X_train_under = training_set_undersampled.drop(target_var, axis =1)
    Y_train_under = training_set_undersampled[target_var]
    
    return X_train_under, Y_train_under


Previous split : 
0 :  415 
1 :  253
New split : 
0 :  200 
1 :  253


In [25]:
X_trainu, Y_trainu = undersample(X_train,Y_train,"Survived",1,200)

Previous split : 
0 :  415 
1 :  253
New split : 
0 :  200 
1 :  253


In [32]:
import binclassfunc as fn

In [33]:
log_reg = LogisticRegression(random_state=0, class_weight = 'balanced')

modelfit = log_reg.fit(X_train,Y_train)

ValueError: could not convert string to float: 'S'

In [ ]:
fn.Model_KPI().compute()

## List model 

In [26]:
## Models being tested 
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB


## log regs
log_reg = LogisticRegression(random_state=0, class_weight = 'balanced')
log_reg_elastic = LogisticRegression(class_weight = 'balanced',penalty= 'l1',solver = 'liblinear')
log_reg_l1 = LogisticRegression(class_weight = 'balanced',penalty= 'elasticnet',solver= 'saga',l1_ratio=0.5) #saga solver works

## svc
svc_lin = svm.SVC(class_weight = 'balanced',kernel = 'linear',probability=True)
svc_poly = svm.SVC(class_weight = 'balanced',kernel = 'poly',probability=True)
svc_rbf = svm.SVC(class_weight = 'balanced',kernel = 'rbf',probability=True)
svc_sigmoid = svm.SVC(class_weight = 'balanced',kernel = 'sigmoid',probability=True)

## NB
multi_NB = MultinomialNB()

## Decision tree
dec_tree = DecisionTreeClassifier(class_weight = 'balanced')
xgboost = XGBClassifier(eval_metric ='auc'#scale_pos_weight = 5
                       ) ###to be explained, its the ratio of imbalance

## Ensemble 
adab = AdaBoostClassifier()
random_forest = RandomForestClassifier(class_weight="balanced")

## LDA
lda = LinearDiscriminantAnalysis() ##svd



## Compute AUC

In [27]:
from sklearn import metrics

def compute_AUC(model,X_test,Y_test):

    yroc = Y_test.as_matrix().reshape(Y_test.as_matrix().size,1)
    predroc = model.predict_proba(X_test)[:,1]
    AUC = metrics.roc_auc_score(yroc, predroc)
    return AUC


## Compute ratios from confusion matrix

In [ ]:
from sklearn.model_selection import cross_val_score


models = [log_reg,
          log_reg_elastic,
          log_reg_l1,
          svc_lin,
          svc_poly,
          svc_rbf,
          svc_sigmoid,
          multi_NB,
          dec_tree,
          xgboost,
          adab, #not adapted
          random_forest,
          lda      
]

undermodel = [multi_NB,
             xgboost,
             adab,
             lda
             ]



CV = 4
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
fitted_models = []
i = 0


### Initializing dataframe to store kPIS

Accuracy_df = pd.DataFrame({"risk":np.arange(0.0, 1.0, 0.005).tolist()})
Precision_df = pd.DataFrame({"risk":np.arange(0.0, 1.0, 0.005).tolist()})
Missclassified_1_df = pd.DataFrame({"risk":np.arange(0.0, 1.0, 0.005).tolist()})
TNR_df = pd.DataFrame({"risk":np.arange(0.0, 1.0, 0.005).tolist()})
TPR_df = pd.DataFrame({"risk":np.arange(0.0, 1.0, 0.005).tolist()})

### initializing list for individual kpis
AUC_list = []

for model in models:
    
    ## report models 
    i = i+1
    model_name = str(str(i) +"-"+ model.__class__.__name__)
    print(model_name)
    
   
    ### verify the data input on undersampling
    if model in undermodel : 
        
        ### We use the undersampled data
        accuracies = cross_val_score(model, Xunder, Yunder, scoring='accuracy', cv=CV)
        
        fitted_model = model.fit(X_train_under,Y_train_under)
        prediction_train = model.predict(X_train_under)
        
        ## compute KPIs
        AUC = compute_AUC(model,X_test_under,Y_test_under)
        Accuracy,Precision_df_1,Missclassified_1,TNR_df_1,TPR_df_1 = compute_KPIs(fitted_model,model_name,X_test_under,Y_test_under)
        
    else : 
        ## cross_val accuracies
        accuracies = cross_val_score(model, X, Y, scoring='accuracy', cv=CV)
        
        fitted_model = model.fit(X_train,Y_train)
        prediction_train = model.predict(X_train)
        
        ## compute KPIs
        AUC = compute_AUC(model,X_test,Y_test)
        Accuracy,Precision_df_1,Missclassified_1,TNR_df_1,TPR_df_1 = compute_KPIs(fitted_model,model_name,X_test, Y_test)
    
    
    #### 
    #Creating DF with results for all models
    ####
    
    Accuracy_df = pd.concat([Accuracy_df, Accuracy],axis = 1)
    Precision_df = pd.concat([Precision_df, Precision_df_1],axis = 1)
    Missclassified_1_df = pd.concat([Missclassified_1_df, Missclassified_1],axis = 1)
    TNR_df = pd.concat([TNR_df, TNR_df_1],axis = 1)
    TPR_df = pd.concat([TPR_df, TPR_df_1],axis = 1)
    
    ###
    # Creating list for indiv KPIs /model
    ###
    
    AUC_list.append(AUC)
    
    number_bads = sum(prediction_train)
   
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy,number_bads))
        
    fitted_models.append(fitted_model)
    
## removing redundant columns of indexes
Accuracy_df= Accuracy_df.loc[:,~Accuracy_df.columns.duplicated()]
Precision_df= Precision_df.loc[:,~Precision_df.columns.duplicated()]
Missclassified_1_df= Missclassified_1_df.loc[:,~Missclassified_1_df.columns.duplicated()]
TNR_df = TNR_df.loc[:,~TNR_df.columns.duplicated()]
TPR_df = TPR_df.loc[:,~TPR_df.columns.duplicated()]

print("\n",i, 'Model Tested')

cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy','number_bads'])